In [1]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils #ci darà tutte le funzioni per mostrare a schermo i disegni, punti linee
mp_pose = mp.solutions.pose #ci darà il nostro modello per la pose estimation


In [2]:
#ci darà l'accesso alla cam : VIDEO FEED
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read() #ret è semplicemente una variabile di ritorno che non useremo effettivamente mentre frame è effettuivamente l'immagine
    cv2.imshow('Mediapipe Feed', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

 # 1. **Fare Detections**

In [3]:
#ci darà l'accesso alla cam : VIDEO FEED
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read() #ret è semplicemente una variabile di ritorno che non useremo effettivamente mentre frame è effettuivamente l'immagine
        
        #Ricoloriamo l'immagine, in modo tale che l'immagine mandata  mediapipe sia nella scala RGB perchè di base il formato usato è BGR
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Facciamo effettivamente detection con il nostro modello
        results = pose.process(image)
        
        #ricoloriamo to BGR poichè open cv funziona con BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  #caratteristche dei punti landmarks
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),#punti
                                  mp_drawing.DrawingSpec(color=(245, 66, 230),thickness=2, circle_radius=2)#linee
                                  )
                
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 2. **Determinare Joints/congiunzioni**

<img src ="img/pose_tracking_full_body_landmarks.png" style = "height:300px">

In [4]:
#ci darà l'accesso alla cam : VIDEO FEED
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read() #ret è semplicemente una variabile di ritorno che non useremo effettivamente mentre frame è effettuivamente l'immagine
        
        #Ricoloriamo l'immagine, in modo tale che l'immagine mandata  mediapipe sia nella scala RGB perchè di base il formato usato è BGR
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Facciamo effettivamente detection con il nostro modello
        results = pose.process(image)
        
        #ricoloriamo to BGR poichè open cv funziona con BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #estraiamo i landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        #Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  #caratteristche dei punti landmarks
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),#punti
                                  mp_drawing.DrawingSpec(color=(245, 66, 230),thickness=2, circle_radius=2)#linee
                                  )
                
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [5]:
len(landmarks)

33

In [6]:
for lndmark in mp_pose.PoseLandmark:
    print(lndmark)

PoseLandmark.NOSE
PoseLandmark.LEFT_EYE_INNER
PoseLandmark.LEFT_EYE
PoseLandmark.LEFT_EYE_OUTER
PoseLandmark.RIGHT_EYE_INNER
PoseLandmark.RIGHT_EYE
PoseLandmark.RIGHT_EYE_OUTER
PoseLandmark.LEFT_EAR
PoseLandmark.RIGHT_EAR
PoseLandmark.MOUTH_LEFT
PoseLandmark.MOUTH_RIGHT
PoseLandmark.LEFT_SHOULDER
PoseLandmark.RIGHT_SHOULDER
PoseLandmark.LEFT_ELBOW
PoseLandmark.RIGHT_ELBOW
PoseLandmark.LEFT_WRIST
PoseLandmark.RIGHT_WRIST
PoseLandmark.LEFT_PINKY
PoseLandmark.RIGHT_PINKY
PoseLandmark.LEFT_INDEX
PoseLandmark.RIGHT_INDEX
PoseLandmark.LEFT_THUMB
PoseLandmark.RIGHT_THUMB
PoseLandmark.LEFT_HIP
PoseLandmark.RIGHT_HIP
PoseLandmark.LEFT_KNEE
PoseLandmark.RIGHT_KNEE
PoseLandmark.LEFT_ANKLE
PoseLandmark.RIGHT_ANKLE
PoseLandmark.LEFT_HEEL
PoseLandmark.RIGHT_HEEL
PoseLandmark.LEFT_FOOT_INDEX
PoseLandmark.RIGHT_FOOT_INDEX


In [7]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.7117854356765747
y: 0.8315130472183228
z: -0.2055463045835495
visibility: 0.9991710782051086

In [8]:
mp_pose.PoseLandmark.LEFT_SHOULDER.value

11

# 3. **Calcolare angoli**

In [2]:
def calculate_angle(a,b,c):
    a=np.array(a) #first questi array dovrebbero essere composti da due punti x,y
    b= np.array(b) #mid
    c = np.array(c) #end

    radians = np.arctan2(c[1]-b[1], c[0]-b[0])- np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360 - angle
    
    return angle

In [10]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y]

In [11]:
shoulder, elbow, wrist

([0.7117854356765747, 0.8315130472183228],
 [0.7805852293968201, 1.2035577297210693],
 [0.8078407049179077, 1.5154774188995361])

In [12]:
calculate_angle(shoulder,elbow,wrist)

174.51683072367058

In [14]:
#ci darà l'accesso alla cam : VIDEO FEED
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read() #ret è semplicemente una variabile di ritorno che non useremo effettivamente mentre frame è effettuivamente l'immagine
        
        #Ricoloriamo l'immagine, in modo tale che l'immagine mandata  mediapipe sia nella scala RGB perchè di base il formato usato è BGR
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Facciamo effettivamente detection con il nostro modello
        results = pose.process(image)
        
        #ricoloriamo to BGR poichè open cv funziona con BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #estraiamo i landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            #otteniamo le coordinate a run-time
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y]
            
            #calcoliamo l'angolo
            angle = calculate_angle(shoulder,elbow,wrist)
            print(angle)
            
            #visualizziamo l'angolo
            cv2.putText(image, str(angle),
                        tuple(np.multiply(elbow,[640,480]).astype(int)), #effetuando la motiplicazione il testo si dispone in maniera corretta in base alla dimensione dell'applicazione, quindi abbiamo le coodinate giuste
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                        )
            
        except:
            pass
        
        #Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  #caratteristche dei punti landmarks
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),#punti
                                  mp_drawing.DrawingSpec(color=(245, 66, 230),thickness=2, circle_radius=2)#linee
                                  )
                
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072367058
174.51683072

KeyboardInterrupt: 

: 

# 4. **Curl Counter**


In [5]:
#ci darà l'accesso alla cam : VIDEO FEED
cap = cv2.VideoCapture(0)

#curl counter variabili
counter = 0
stage = None

#setup mediapipe
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read() #ret è semplicemente una variabile di ritorno che non useremo effettivamente mentre frame è effettuivamente l'immagine
        
        #Ricoloriamo l'immagine, in modo tale che l'immagine mandata  mediapipe sia nella scala RGB perchè di base il formato usato è BGR
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Facciamo effettivamente detection con il nostro modello
        results = pose.process(image)
        
        #ricoloriamo to BGR poichè open cv funziona con BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #estraiamo i landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            #otteniamo le coordinate a run-time
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y]
            
            #calcoliamo l'angolo
            angle = calculate_angle(shoulder,elbow,wrist)
            
            #visualizziamo l'angolo
            cv2.putText(image, str(angle),
                        tuple(np.multiply(elbow,[640,480]).astype(int)), #effetuando la motiplicazione il testo si dispone in maniera corretta in base alla dimensione dell'applicazione, quindi abbiamo le coodinate giuste
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                        )
            
            #curl counter logica
            if angle > 160: #treshold indicativo
                stage = "down"
            if angle < 30 and stage == 'down':
                stage = "up"
                counter +=1
                print(counter)
            
        except:
            pass
        
        #Render curl counter
        #setup status box
        cv2.rectangle(image, (0,0), (275,73), (245,117,16), -1)
        
        #Inserisco le rep 
        cv2.putText(image,'REPS', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        #Inserisco le rep 
        cv2.putText(image,'STAGE', (65,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(stage),
                    (60,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        #Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  #caratteristche dei punti landmarks
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),#punti
                                  mp_drawing.DrawingSpec(color=(245, 66, 230),thickness=2, circle_radius=2)#linee
                                  )
                
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
